In [35]:
import os
import re
import json
import spacy
import gensim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, wordpunct_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from tqdm import tqdm
from collections import Counter
from PIL import Image
from string import punctuation

# 1. Сбор данных

Собрать данные со всех файлов: имена героев и их реплики. Файлы все собраны в одной папке, в каждом файле имя героя, потом двоеточие и после реплика. Кроме это были лишние данные в виде описания сцены и действий персонажей, которые я удалила

In [40]:
re_character = re.compile(r'(.+?):')  # найти имя персонажа
re_text = re.compile(r': (.+)')  # найти реплику
re_episode = re.compile(r'(\w+?)\.')  # выделить название эпизода = название файла
re_del = re.compile(r'\[.+?\]')  # удаление ненужных данных
re_title = re.compile(r'[A-Z][a-z]*')  # разделение название эпизода пробелами, так как слова написаны слитно

In [3]:
def define_character(lines, file):
    character_texts = []
    episode = re_episode.match(file)[1]
    for n, line in enumerate(lines):
        line = re_del.sub('', line).replace('\xa0', ' ')
        if re_character.match(line) and re_text.search(line):
            character = re_character.match(line)[1]
            text = re_text.search(line)[1]
            character_texts.append(
                {'character': character, 
                 'episode': episode, 
                 'line': text}
            )
    return character_texts

In [36]:
characters_table = pd.DataFrame(columns=['character', 'episode', 'line'])
for root, dirs, files in os.walk('../SpongeBob_SquarePants_Transcripts'):
    for file in tqdm(files):
        path = os.path.join(root, file)
        with open(path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        character_texts = define_character(lines, file)
        characters_table = characters_table.append(character_texts, ignore_index=True)

100%|████████████████████████████████████████| 393/393 [00:00<00:00, 702.48it/s]


In [37]:
characters_table = characters_table.drop(index=32911)

In [38]:
def normalize_title(title):
    return ' '.join(re_title.findall(title))

In [41]:
characters_table['episode'] = characters_table['episode'].apply(normalize_title)

# 2. Анализ данных

Какие поинты можно отметить:
- длина реплики каждого персонажа из основных
- самое часто слово у каждого / во всём сериале
- серия с наибольшим количеством реплик
- облака слов каждого персонажа из основных
- можно для прикола просто факты накидать (спарсить с сайта какого-нибудь?:))

Я выделила 8 основных персонажей мультсериала, создала словарь, куда записывала полученные данные по каждому персонажу, а также создала отдельную таблицу, чтобы анализировать

In [42]:
char_info = {'SpongeBob': {},
            'Patrick': {},
            'Mr. Krabs': {},
            'Squidward': {},
            'Plankton': {},
            'Sandy': {},
            'Karen': {},
            'Gary': {}}

In [43]:
main_char = ['SpongeBob', 'Patrick', 'Mr. Krabs', 'Squidward', 'Plankton', 'Sandy', 'Karen', 'Gary']
main_char_table = characters_table.copy()[characters_table['character'].isin(main_char)]

### Длина реплики

Я посчитала медианную длину реплики каждого их основных персонажей

In [45]:
main_char_table['len_line'] = main_char_table['line'].apply(lambda x: len(x.split()))

In [46]:
cnt_len_table = (main_char_table
                 .groupby('character')['len_line']
                 .agg('median')
                 .reset_index(name='count'))

In [47]:
for i, row in cnt_len_table.iterrows():
    char_info[row['character']]['length'] = row['count']

### Вордклауды

Для каждого из основных персонажей я создала облако слов. 

Для этого понадобилось:
1. С помощью модуля nltk очистить и лемматизировать текст
2. Найти маски для каждого из персонажей, чтобы сделать фигурные облака слов

In [48]:
all_lines = main_char_table.groupby('character')['line'].agg(list).reset_index()

In [49]:
found_sw = ['go', 'get', 'oh', 
            'well', 'like', 'come', 
            'look', 'know', 'see', 
            'hey', 'na', 'one', 'two']
sw = stopwords.words('english') + found_sw
lemmatizer = WordNetLemmatizer()
# для корректной работы лемматизатора понадобилось перевести
# выражение части речи, которая получалась в результате функции pos_tag
# в выражение части речи, которую принимала на вход функция lemmatize
nltk_to_pos = {'ADV': 'r', 'NOUN': 'n', 'VERB': 'v', 'ADJ': 'a'}

In [50]:
def for_model(lines):
    lemmas = []
    new_sent = []
    for line in lines:
        sentences = sent_tokenize(line)
        for sent in sentences:
            words = [word for word in word_tokenize(sent) if word.isalpha()]
            for word in words:
                pos = pos_tag([word], tagset='universal')[0][1]
                if pos in ['VERB', 'NOUN', 'ADV', 'ADJ']:
                    word = lemmatizer.lemmatize(word.lower(), pos=nltk_to_pos[pos])
                else:
                    word = lemmatizer.lemmatize(word.lower())
                if word not in sw:
                    lemmas.append(word)
    return lemmas

In [51]:
all_lines['lemmas'] = all_lines['line'].apply(for_model)

In [55]:
masks = {'SpongeBob': '../masks/spongebob_mask.png', 
         'Patrick': '../masks/patrick_mask.png',
         'Mr. Krabs': '../masks/mrkrabs_mask.png',
         'Sandy': '../masks/sandy_mask.png',
         'Squidward': '../masks/squidward_mask.png',
         'Karen': '../masks/karen_mask.png',
         'Plankton': '../masks/plankton_mask.png',
         'Gary': '../masks/gary_mask.png'
        }
for i, name in enumerate(all_lines['character'].unique()):
    mask = np.array(Image.open(masks[name]))
    text = ' '.join(all_lines[all_lines['character'] == name]['lemmas'][i])
    wordcloud = WordCloud(
        background_color ='white',
        width = 800,
        height = 800,
        mask = mask).generate(text)
    wordcloud.to_file(f'{masks[name]}_cloud.png')
    char_info[name]['wordcloud'] = f'{masks[name][1:]}_cloud.png'

### Наиболее частые слова персонажа

Для каждого из основных персонажей я выделила топ-5 слов, которые они используют. Неудивительно, что у каждого из них (кроме Гэри, сответственно) одно из топ-5 слов - это SpongeBob

In [56]:
def frequent_word(words):
    count_words = Counter(words)
    return count_words.most_common(5)

In [57]:
all_lines['frequent'] = all_lines['lemmas'].apply(frequent_word)

In [58]:
for i, row in all_lines.iterrows():
    char_info[row['character']]['most_words'] = row['frequent']

### Количество серий, в которых встречается персонаж

Последнее, что я выяснила про персонажа, это количество эпизодов, в которых он встретился

In [59]:
def count_episodes(episodes):
    return len(episodes)

In [60]:
all_episodes = main_char_table.groupby('character')['episode'].unique().reset_index(name='all_episodes')

In [61]:
all_episodes['count_episodes'] = all_episodes['all_episodes'].apply(count_episodes)

In [63]:
all_episodes.sort_values(['count_episodes'])

,character,all_episodes,count_episodes
1,Karen,"[Karen, A Cabininthe Kelp, Goo Goo Gas, Grandm...",55
5,Sandy,"[Oral Report, Stuckonthe Roof, A Cabininthe Ke...",96
4,Plankton,"[The Krusty Slammer, The Great Patty Caper, Ka...",97
0,Gary,"[Penny Foolish, Sentimental Sponge, Bummer Vac...",110
2,Mr. Krabs,"[Lighthouse Louie, Penny Foolish, Barnacle Fac...",262
3,Patrick,"[Oral Report, Sentimental Sponge, Fun Sized Fr...",270
7,Squidward,"[Penny Foolish, Oral Report, The Krusty Slamme...",295
6,SpongeBob,"[Lighthouse Louie, Penny Foolish, Barnacle Fac...",389


In [64]:
for i, row in all_episodes.iterrows():
    char_info[row['character']]['count_episodes'] = row['count_episodes']

In [65]:
with open('char_info.json', 'w') as f:
    f.write(json.dumps(char_info))

### Количество реплик в эпизоде

In [66]:
lines_per_episode = (characters_table
                     .groupby('episode')['line']
                     .agg('count')
                     .reset_index(name='lines_cnt'))

### Количество персонажей в эпизоде

In [67]:
character_per_episode = (characters_table
                     .groupby('episode')['character']
                     .agg('unique')
                     .reset_index(name='unique_char'))

In [68]:
character_per_episode['unique_char_cnt'] = character_per_episode['unique_char'].apply(len)

In [71]:
episode_stat = {'lines_cnt': {}, 'chatacters_cnt': {}}

In [70]:
episodes = character_per_episode.join(lines_per_episode.set_index('episode'), on='episode')

In [72]:
# больше всего персонажей
a = episodes.sort_values(['unique_char_cnt'], ascending=False)[:3]
# меньше всего персонажей
b = episodes.sort_values(['unique_char_cnt'], ascending=True)[:3]
for i, row in a.iterrows():
    episode_stat['chatacters_cnt'][row['episode']] = [row['unique_char_cnt']]
for i, row in b.iterrows():
    episode_stat['chatacters_cnt'][row['episode']] = [row['unique_char_cnt'], row['unique_char'].tolist()]

In [73]:
# больше всего реплик
a = episodes.sort_values(['lines_cnt'], ascending=False)[:3]
# меньше всего реплик
b = episodes.sort_values(['lines_cnt'], ascending=True)[:3]
for i, row in a.iterrows():
    episode_stat['lines_cnt'][row['episode']] = row['lines_cnt']
for i, row in b.iterrows():
    episode_stat['lines_cnt'][row['episode']] = row['lines_cnt']

In [74]:
with open('episodes.json', 'w') as f:
    f.write(json.dumps(episode_stat))